# Part 0: Dataloader and Visualizations

In [1]:
import torch
import wandb
import scipy.io

import numpy as np

import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
from torchvision import transforms, datasets
from torch.utils.data import DataLoader

from voc_dataset import VOCDataset

from PIL import Image

from utils import *

USE_WANDB = True

## Editing the Dataloader
The first part of the assignment involves editing the dataloader so that we can access bounding-box proposals as well as the ground-truth bounding boxes. The ground truth bounding box can be accessed through the VOC Dataset annotations itself. Unsupervised bounding box proposals are obtained through methods such as [Selective Search](https://ivi.fnwi.uva.nl/isis/publications/2013/UijlingsIJCV2013/UijlingsIJCV2013.pdf).

Since Selective Search is slow to run on each image, we have pre-computed the bounding box proposals. You should be able to access the `.mat` files using `scipy.io.loadmat('file.mat')`. Feel free to experiment with the data in the files to figure out the number of proposals per image, their scores, etc.

Your task is to change the dataloader to obtain the ground-truth bounding boxes, as well as the proposed bounding boxes for each image. Returning a dictionary would be convenient here. For the bounding boxes, using the relative positions is usually a better idea since they are invariant to changes in the size of the image.

In [2]:
# Load the Dataset - items at a particular index can be accesed by usual indexing notation (dataset[idx])
dataset = VOCDataset('trainval', top_n=10, image_size=224)

In [3]:
#TODO: get the image information from index 2020
idx = 2020

input = dataset.__getitem__(idx)
original_image = tensor_to_PIL(input['image'])
gt_labels = input['gt_classes']
gt_boxes = input['gt_boxes']
rois = input['rois'] 

In [4]:
#gt_boxes, input['gt_classes'], input['label']
class_id_to_label = dict(enumerate(dataset.CLASS_NAMES))
class_id_to_label

{0: 'aeroplane',
 1: 'bicycle',
 2: 'bird',
 3: 'boat',
 4: 'bottle',
 5: 'bus',
 6: 'car',
 7: 'cat',
 8: 'chair',
 9: 'cow',
 10: 'diningtable',
 11: 'dog',
 12: 'horse',
 13: 'motorbike',
 14: 'person',
 15: 'pottedplant',
 16: 'sheep',
 17: 'sofa',
 18: 'train',
 19: 'tvmonitor'}

## Wandb Init and Logging
Initialize a Weights and Biases project, and convert the image tensor to a PIL image and plot it (check `utils.py` for helper functions).

You can use [this](https://docs.wandb.ai/library/log) as a reference for logging syntax.

In [5]:
if USE_WANDB:
    wandb.init(project="vlr2", reinit=True)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: akshayantony12 (use `wandb login --relogin` to force relogin)
/home/akshay/anaconda3/envs/materials2/lib/python3.9/site-packages/IPython/html.py:12: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  warn("The `IPython.html` package has been deprecated since IPython 4.0. "


In [6]:
img = wandb.Image(original_image)
wandb.log({"org": img})

See this block as an example of plotting the ground truth box for an image.

In [7]:
class_id_to_label = dict(enumerate(dataset.CLASS_NAMES))
gt_boxes = gt_boxes
gt_labels = gt_labels
img = wandb.Image(original_image, boxes={
    "predictions": {
        "box_data": get_box_data(gt_labels, gt_boxes),
        "class_labels": class_id_to_label,
    },
})
wandb.log({"gt label": img})

Check the `get_box_data` function in `utils.py` and understand how it is being used. Log the image with the GT bounding box on wandb.
After, this you should be able to easily plot the top 10 bounding proposals as well.

In [8]:
nums = range(len(rois)) # placeholder for names of proposals
#TODO: plot top ten proposals (of bounding boxes)
img_proposals = wandb.Image(original_image, boxes={
                "predictions":{
                    "box_data": get_box_data(nums, rois),
                    "class_labels": class_id_to_label,
                },
})
wandb.log({"proposal": img_proposals})